In [5]:
import numpy as np
import pandas as pd
import warnings

# from pandas.core.common import SettingWithCopyWarning
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

province_dict = {
    "11": "北京市",
    "12": "天津市",
    "13": "河北省",
    "14": "山西省",
    "15": "内蒙古自治区",
    "21": "辽宁省",
    "22": "吉林省",
    "23": "黑龙江省",
    "31": "上海市",
    "32": "江苏省",
    "33": "浙江省",
    "34": "安徽省",
    "35": "福建省",
    "36": "江西省",
    "37": "山东省",
    "41": "河南省",
    "42": "湖北省",
    "43": "湖南省",
    "44": "广东省",
    "45": "广西壮族自治区",
    "46": "海南省",
    "50": "重庆市",
    "51": "四川省",
    "52": "责州省",
    "53": "云南省",
    "54": "西藏自治区",
    "61": "陕西省",
    "62": "甘肃省",
    "63": "青海省",
    "64": "宁夏回族自治区",
    "65": "新疆维吾尔自治区",
    "81": "香港特别行政区"
}

charging_power_type_dict = {
    0: "交流",
    1: "直流"
}

pile_file = 'D:\charging_evcs_20230110\operator_charging_pile_202301101756.csv'

pile_column_names = [
    "id","station_id","station_name","pile_id","pile_name",
    "equipment_type","equipment_model","charging_port_num","operator_id","operator_name",
    "longitude","latitude","dept_code","producer_name","producer_time",
    "charging_power","charging_model_no","is_deleted","create_time","create_user","create_dept",
    "update_user","update_time","status","is_charge_timeout","timeout_free_time",
    "timeout_free_price","upper_limit_amount","timeoutfree_time_desc","equipment_has_lock"
]

pile_data_types = dict.fromkeys(pile_column_names, np.string_)
pile_data_types['id'] = np.int64
pile_data_types['equipment_type'] = np.int32
pile_data_types['charging_port_num'] = np.int32
pile_data_types['is_deleted'] = np.int32
pile_data_types['status'] = np.int32


station_file = 'D:\charging_evcs_20230110\operator_charging_station_202301101800.csv'

station_column_names = [
    "id","station_id","charging_station_name","equipment_owner_id","station_address",
    "pile_number","platform_station_no","operator_id","operator_name","station_country_code",
    "province_code","city_code","longitude","latitude","station_type",
    "station_status","car_number","station_guide","station_tel","service_tel",
    "service_place","apply_car","station_photo","pay_model","space_description",
    "service_fee_brief","parking_fee_brief","charging_charge_brief","work_hours_description","support_order",
    "remark","is_deleted","create_time","create_user","create_dept",
    "update_user","update_time","status","area_code","park_fee_free",
    "station_has_lock"
]

station_data_types = dict.fromkeys(station_column_names, np.string_)
station_data_types['id'] = np.int64
station_data_types['car_number'] = np.int32
station_data_types['is_deleted'] = np.int32
station_data_types['status'] = np.int32

In [6]:
df_pile = pd.read_csv(pile_file, encoding="utf-8", names=pile_column_names, skiprows=1, dtype=pile_data_types)
df_pile = df_pile[['id', 'station_id', 'charging_power']]

df_pile['charging_power'] = df_pile['charging_power'].astype(np.float32)
# df_pile.describe()

df_station = pd.read_csv(station_file, encoding="utf-8", names=station_column_names, skiprows=1, dtype=station_data_types)
df_station = df_station[['station_id', 'province_code', 'city_code', 'station_type']]
# df_station.describe()

df = df_pile.merge(df_station, on="station_id")
# df.describe()

In [7]:
df.insert(0, 'charging_power_type', value="")

# charing_power_type
df["charging_power_type"].loc[df["charging_power"] <= 7.0] = charging_power_type_dict[0]
df["charging_power_type"].loc[df["charging_power"] > 7.0] = charging_power_type_dict[1]

df.insert(0, 'province_name', value="")

for k in province_dict:
    df["province_name"].loc[df["province_code"] == k] = province_dict[k]

C:\Users\sheng.xu\AppData\Local\Temp\ipykernel_29412\2896206882.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["charging_power_type"].loc[df["charging_power"] <= 7.0] = charging_power_type_dict[0]
C:\Users\sheng.xu\AppData\Local\Temp\ipykernel_29412\2896206882.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["charging_power_type"].loc[df["charging_power"] > 7.0] = charging_power_type_dict[1]
C:\Users\sheng.xu\AppData\Local\Temp\ipykernel_29412\2896206882.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [8]:
grouped = df.groupby(['province_name', 'charging_power_type'])
out = pd.DataFrame(grouped.count()["id"])

out.to_csv('d:\桩分布.csv', encoding='utf-8-sig')